## Libraries, API Key

Don't forget to insert your Google API key

In [2]:
import googlemaps
import numpy as np
import os
import pandas as pd
from scipy.spatial.distance import cdist
import re

#GOOGLE_API_KEY = ""
#gmaps = googlemaps.Client(key = GOOGLE_API_KEY) 

## Constants

Import folder - the one with files like Leuven_aeds.csv (we have to make names more clear in final version)
Export folder - any folder you want

RATIO_EMERGENCY_NORMAL = 0.65 - this means that ambulance is 35% faster than a regular car for the same distance

In [3]:
import_folder_path = "C:/Users/lucas/OneDrive/OTHER/GitHub/MDA-Project/Data"
export_folder_path = "/Users/Jovan/Desktop/MDA_Project/DistanceMatrices"

cities = ["Antwerpen", "Brugge", "Brussels", "Charleroi", "Gent", "Leuven", "Liege", "Oostende"]

RATIO_EMERGENCY_NORMAL = 0.65

## Functions

In [4]:
# Adjusting coordinates so they suit directions() function

def format_coordinates(latitude, longitude):
    return f"{latitude}, {longitude}"

# Function to extract latitude and longitude from geometry column
def extract_coordinates(geometry):
    match = re.match(r'POINT \(([^ ]+) ([^ ]+)\)', geometry)
    if match:
        longitude, latitude = match.groups()
        return float(latitude), float(longitude)
    else:
        raise ValueError("Invalid geometry format")

In [5]:
# Calculates distance matrix for given sets of origin and destination coordinates

def get_distance_matrix(origins, destinations, mode, coef = 1):
    num_origins = len(origins)
    num_destinations = len(destinations)
    result_matrix = np.zeros((num_origins, num_destinations))

    confirmation = input(f"This will initialize {num_origins*num_destinations} calculations. Are you sure? (yes/no): ")
    if confirmation == "yes":
        for i in range(0, num_origins):
            for j in range(0, num_destinations):
                orig = format_coordinates(origins.iloc[i, 0], origins.iloc[i, 1])
                dest = format_coordinates(destinations.iloc[j, 0], destinations.iloc[j, 1])
                print("Calculating pair: [", i, ",", j, "]")
                result_matrix[i][j] = gmaps.directions(orig, dest, mode)[0]['legs'][0]['duration']['value']
                print("Time:", result_matrix[i][j], "\n")

    return(result_matrix * coef)

## Calculating Distance Matrix

In [8]:
# Number of closest AEDs to consider
closest_aeds = 7

for city in cities:
    # Import data on the city
    os.chdir(import_folder_path)
    aeds = pd.read_csv(city + "_aeds.csv")
    cards = pd.read_csv(city + "_cards.csv")
    new_aeds = pd.read_csv(city + "_new_aeds.csv")
    print("Importing " + city + " successful.")
    
    # Make 'mandatory' columns for aeds and vehicles
    aeds['public'] = aeds['public'].fillna(0)
    aeds['public'] = ~aeds['public'].astype(bool)
    aeds = aeds.rename(columns={'public': 'mandatory'})

    aeds_coordinates = aeds[['latitude', 'longitude']]
    
    # Extract latitude and longitude from geometry column in new_aeds
    new_aeds[['latitude', 'longitude']] = new_aeds['geometry'].apply(
        lambda x: pd.Series(extract_coordinates(x))
    )
    new_aeds_coordinates = new_aeds[['latitude', 'longitude']]
    
    cards_coordinates = cards[['latitude', 'longitude']]
    
    new_aeds['mandatory'] = 0
    print("Made both 'mandatory' columns.\n")
    
    # Combine aeds_coordinates and new_aeds_coordinates
    combined_aeds_coordinates = pd.concat([aeds_coordinates, new_aeds_coordinates], ignore_index=True)
    
    # Combine mandatory columns
    combined_mandatory = pd.concat([aeds['mandatory'], new_aeds['mandatory']], ignore_index=True)
    
    # Calculate the Euclidean distance matrix
    distance_matrix = cdist(combined_aeds_coordinates, cards_coordinates, metric='euclidean')
    
    # Process the distance matrix to mark the closest AEDs
    closest_matrix = np.zeros_like(distance_matrix)
    for col in range(distance_matrix.shape[1]):
        col_indices = np.argsort(distance_matrix[:, col])[:closest_aeds]
        closest_matrix[col_indices, col] = 1
    
    # Create MultiIndex for rows and columns
    aeds_index = pd.MultiIndex.from_tuples(combined_aeds_coordinates.to_records(index=False).tolist(),
                                           names=['AED Latitude', 'AED Longitude'])
    cards_index = pd.MultiIndex.from_tuples(cards_coordinates.to_records(index=False).tolist(),
                                            names=['Card Latitude', 'Card Longitude'])
    
    # Create DataFrame with MultiIndex and values from closest_matrix
    closest_matrix_df = pd.DataFrame(closest_matrix, index=aeds_index, columns=cards_index)
    
    # Print the resulting matrix
    print(f"Closest AED matrix for {city}:")
    print(closest_matrix_df)
    
    print(f"Processed {city} successfully.\n")

Importing Antwerpen successful.
Made both 'mandatory' columns.



C:\Users\lucas\AppData\Local\Temp\ipykernel_31180\622861697.py:34: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  combined_mandatory = pd.concat([aeds['mandatory'], new_aeds['mandatory']], ignore_index=True)


Closest AED matrix for Antwerpen:
Card Latitude              51.23266         51.24848         51.28273          \
Card Longitude              4.44440 4.44440  4.43869 4.43869  4.42134 4.42134   
AED Latitude AED Longitude                                                      
51.196733    4.429455           0.0     0.0      0.0     0.0      0.0     0.0   
51.227416    4.404354           0.0     0.0      0.0     0.0      0.0     0.0   
51.229230    4.437683           0.0     0.0      0.0     0.0      0.0     0.0   
51.186448    4.371285           0.0     0.0      0.0     0.0      0.0     0.0   
51.203595    4.420659           0.0     0.0      0.0     0.0      0.0     0.0   
...                             ...     ...      ...     ...      ...     ...   
51.250089    4.446447           0.0     0.0      0.0     0.0      0.0     0.0   
51.291999    4.378724           0.0     0.0      0.0     0.0      0.0     0.0   
51.293105    4.443447           0.0     0.0      0.0     0.0      0.0     0

C:\Users\lucas\AppData\Local\Temp\ipykernel_31180\622861697.py:34: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  combined_mandatory = pd.concat([aeds['mandatory'], new_aeds['mandatory']], ignore_index=True)
C:\Users\lucas\AppData\Local\Temp\ipykernel_31180\622861697.py:34: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  combined_mandatory = pd.concat([aeds['mandatory'], new_aeds['mandatory']], ignore_index=True)


Closest AED matrix for Brussels:
Card Latitude              50.861990          50.775180 50.830394           \
Card Longitude              4.415180 4.415180  4.338500  4.255696 4.255696   
AED Latitude AED Longitude                                                   
50.842822    4.384308            0.0      0.0       0.0       0.0      0.0   
50.883018    4.333151            0.0      0.0       0.0       0.0      0.0   
50.880253    4.325061            0.0      0.0       0.0       0.0      0.0   
50.874774    4.331500            0.0      0.0       0.0       0.0      0.0   
50.832142    4.327469            0.0      0.0       0.0       0.0      0.0   
...                              ...      ...       ...       ...      ...   
50.865590    4.326667            0.0      0.0       0.0       0.0      0.0   
50.850945    4.307144            0.0      0.0       0.0       0.0      0.0   
50.834742    4.266173            0.0      0.0       0.0       1.0      1.0   
50.888822    4.411036          

...

C:\Users\lucas\AppData\Local\Temp\ipykernel_31180\622861697.py:34: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  combined_mandatory = pd.concat([aeds['mandatory'], new_aeds['mandatory']], ignore_index=True)
C:\Users\lucas\AppData\Local\Temp\ipykernel_31180\622861697.py:34: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  combined_mandatory = pd.concat([aeds['mandatory'], new_aeds['mandatory']], ignore_index=True)


Closest AED matrix for Gent:
Card Latitude              51.079042 51.026948          51.054924           \
Card Longitude              3.711116  3.691807 3.691807  3.689833 3.689833   
AED Latitude AED Longitude                                                   
51.010247    3.702113            0.0       0.0      0.0       0.0      0.0   
51.039030    3.723701            0.0       0.0      0.0       0.0      0.0   
             3.723701            0.0       0.0      0.0       0.0      0.0   
             3.723701            0.0       0.0      0.0       0.0      0.0   
             3.723701            0.0       0.0      0.0       0.0      0.0   
...                              ...       ...      ...       ...      ...   
51.034833    3.639026            0.0       0.0      0.0       0.0      0.0   
51.079051    3.683985            0.0       0.0      0.0       0.0      0.0   
51.049413    3.764106            0.0       0.0      0.0       0.0      0.0   
51.163084    3.799886            0.

C:\Users\lucas\AppData\Local\Temp\ipykernel_31180\622861697.py:34: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  combined_mandatory = pd.concat([aeds['mandatory'], new_aeds['mandatory']], ignore_index=True)


Closest AED matrix for Liege:
Card Latitude              50.63012 50.64876 50.62105         50.64579  \
Card Longitude              5.53129  5.48225  5.49337 5.49337  5.50684   
AED Latitude AED Longitude                                               
50.645622    5.573620           0.0      0.0      0.0     0.0      0.0   
50.646806    5.571031           0.0      0.0      0.0     0.0      0.0   
50.638018    5.573975           0.0      0.0      0.0     0.0      0.0   
50.643498    5.583673           0.0      0.0      0.0     0.0      0.0   
50.643429    5.571823           0.0      0.0      0.0     0.0      0.0   
...                             ...      ...      ...     ...      ...   
50.662768    5.572178           0.0      0.0      0.0     0.0      0.0   
50.646872    5.591992           0.0      0.0      0.0     0.0      0.0   
50.649245    5.454326           0.0      0.0      0.0     0.0      0.0   
50.615268    5.569098           0.0      0.0      0.0     0.0      0.0   
50.61920

C:\Users\lucas\AppData\Local\Temp\ipykernel_31180\622861697.py:34: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  combined_mandatory = pd.concat([aeds['mandatory'], new_aeds['mandatory']], ignore_index=True)
